In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

import stumpy

In [ ]:
steam_df = pd.read_csv("/kaggle/input/usd-rub-timeseries/UsdRubTime.csv",delimiter =';')
steam_df['Data']
pd.to_datetime(steam_df['Data'],dayfirst='True')
steam_df.head()

In [ ]:
y = steam_df['Close']
x = steam_df.index

In [ ]:
m = 640
matrix_profile = stumpy.stump(y, m)

In [ ]:
best_motif = matrix_profile_df[matrix_profile_df['profile'] == matrix_profile_df['profile'].min()]
best_motifmatrix_profile_df = pd.DataFrame(matrix_profile, columns=['profile', 'profile index', 'left profile index', 'right profile index'])

matrix_profile_df.tail(10)

In [ ]:
best_motif = matrix_profile_df[matrix_profile_df['profile'] == matrix_profile_df['profile'].min()]
best_motif

In [ ]:
profile_df = matrix_profile_df[['profile']]

fig, ax = plt.subplots(2, figsize=(16,8), sharex=True)
g1 = sns.lineplot(y=y, x=x, ax=ax[0])
g2 = sns.lineplot(data=profile_df, ax=ax[1])

for idx in best_motif.index.to_list():
    g1.axvline(x=idx, color="green")
    g2.axvline(x=idx, color="green")
    rect = Rectangle((idx, 0), m, 40, facecolor="lightgrey")
    g1.add_patch(rect)

Upload txt file OpenSingal format, transform to dataframe pandas format 

In [2]:
import pandas as pd
import re

# Функция для загрузки данных из файла
def load_opensignals_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Извлечение метаданных
    metadata = {}
    for line in lines:
        if line.startswith("#"):
            # Убираем символ # и парсим JSON
            json_str = line.strip()[1:]
            metadata.update(eval(json_str))
        else:
            break  # Прерываем цикл, когда доходим до данных

    # Извлечение данных
    data = []
    for line in lines:
        if not line.startswith("#"):
            values = list(map(int, line.strip().split()))
            data.append(values)

    return metadata, data

# Функция для преобразования данных в DataFrame
def convert_to_dataframe(metadata, data):
    # Извлечение необходимых параметров
    sampling_rate = metadata.get("sampling rate", 1000)  # Частота дискретизации
    label = metadata.get("label", ["A2"])[0]  # Метка канала (например, "A2")

    # Создание DataFrame
    df = pd.DataFrame(data, columns=metadata["column"])

    # Добавление столбца с временем
    df["time"] = df.index / sampling_rate

    # Переименование столбца с амплитудой
    amplitude_column = metadata["column"][-1]  # Последний столбец — амплитуда
    df.rename(columns={amplitude_column: "amplitude"}, inplace=True)

    # Выбор нужных столбцов
    df = df[["time", "amplitude"]]
    df.insert(0, "label", label)  # Добавляем столбец с меткой

    return df



In [ ]:
file_path = "SampleECG.txt"  # Укажите путь к вашему файлу
metadata, data = load_opensignals_file(file_path)
df = convert_to_dataframe(metadata, data)

# Сохранение в CSV
output_csv_path = "output_ecg.csv"
df.to_csv(output_csv_path, index=False)

print(f"Данные сохранены в файл: {output_csv_path}")
print(df.head())  # Вывод первых строк DataFrame для проверки